# CONKAT-seq for multigenomic PAC library mapping

#### generates networks of biosynthetic domains and track their physical location in the PAC library
#### input : demultiplexed AD/KS reads from 1) genomes 2) plate_pools=megapool 3) well_pools
#### output : graphml file

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import pandas as pd
import re
import sys 
import os
from os.path import isfile, join
from Bio import SeqIO
from Bio import SeqFeature
import networkx as nx
import numpy as np
from collections import Counter
import itertools
import shutil
import os
import subprocess
import random
import string
from collections import defaultdict
from itertools import combinations,combinations_with_replacement
from scipy import stats
import tempfile
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
import csv
import operator
import networkx.algorithms.community.centrality
import networkx.algorithms.centrality

In [2]:
def log(s):
    print(s)
    
def makeFasta(headers,seqs,outputfile):
   
    headers =  ['>'+x if x[0] != '>' else x for x in headers]
    fastaList = [val for pair in zip(headers, seqs) for val in pair]
    fastaOutputFile = outputfile
    fileHandle = open(fastaOutputFile, 'w')
    for item in fastaList:
        fileHandle.write("%s\n" % item)


def calc_fisher(a,b,c,d):
    oddsratio, pvalue = stats.fisher_exact([[a,b],[c,d]])
    return oddsratio,pvalue     

def execute(command,screen=False):
    msg = ''
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    # Poll process for new output until finished
    while True:
        nextline = process.stdout.readline()
        if nextline == '' and process.poll() is not None:
            break
        msg = msg + nextline
        if screen:
            sys.stdout.write(nextline)
            sys.stdout.flush()

    output = process.communicate()[0]
    exitCode = process.returncode
    if (exitCode == 0):
        return msg
    else:
        print("%s --> exitcode: %s --> %s" % (command, exitCode, output))

def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
        
def is_neighbour(pair):
    dif = abs(int(pair[0]%384)-int(pair[1]%384))
    if dif in [1,15,16,17]:
        return True
    else:
        return False
    
def toPos(well):
    well = (well-1)%384
    col = (well / 16) + 1
    row = chr((well % 16)+65)
    return (row+str(col))


In [3]:
def derep_cat(INPATH,OUTPATH, sample_name, 
                            remove_files=False,
                           threads=40, verbose=False,run=True):
    
    merged_sorted_output_file = OUTPATH + sample_name + '_SORTED.fna'
    
    if os.path.isfile(merged_sorted_output_file):
            log('File exists --> %s' % merged_sorted_output_file)
    else:
        merged_output_file = OUTPATH + sample_name + '.fna'   
        #make directories for processed files 
        output_subfolders = ['trim/','derep/']
        for dirname in output_subfolders:
                ensure_dir(OUTPATH + dirname)

        demux_files = [f for f in os.listdir(INPATH) if isfile(join(INPATH, f))]
        for i,filename in enumerate(demux_files):
            if i % 100 == 0 :
                print('%s files processed...' % i)

            #trim
            input_full_path = INPATH + filename
            sufix = os.path.splitext(filename)[1]
            output_filenmae = filename.replace(sufix,'.trim' + sufix)
            output_full_path = OUTPATH + 'trim/' + output_filenmae
            strip_left="21"
            truncate="200"
            cmd = ('vsearch '
                   '--threads %s ' 
                   '--fastx_filter %s '
                   '--fastaout %s '
                   '--fastq_stripleft %s '
                   '--fastq_trunclen %s '
                   % (threads,input_full_path,output_full_path,strip_left,truncate)
                  )
            if verbose:
                print('\n')
                print(cmd)

            if run:
                execute(cmd,screen=verbose)

            #dereplicate
            input_file = output_full_path
            output_filenmae = output_filenmae.replace(sufix,'.derep' + sufix)
            output_full_path = OUTPATH + 'derep/' + output_filenmae

            cmd = ('vsearch '
                   '--threads %s ' 
                   '--derep_fulllength %s '
                   '--strand plus '
                   '--output %s '
                   '-sizeout '
                   '--fasta_width 0'
                   % (threads,input_file,output_full_path)
                  )
            if verbose:
                print('\n')
                print(cmd)
            if run:
                execute(cmd,screen=verbose)

        #merge 
        cmd = 'cat %s/derep/* > %s' %(OUTPATH,merged_output_file)
        if verbose:
            print('\n')
            print(cmd)

        if run:
            execute(cmd,screen=verbose)
    
        
        if remove_files:
            for dirname in output_subfolders:
                try:
                    shutil.rmtree(OUTPATH + dirname)
                except:
                    print('Unable to remove %s...' % (OUTPATH+dirname))
                
        
        
        #sortbylength (note: equal length reads are sorted by number of reads )        
        cmd = ('vsearch '
               '--threads %s ' 
               '--sortbylength %s '
               '--output %s '
               % (threads,merged_output_file,merged_sorted_output_file)
              )
        
        if verbose:
            print('\n')
            print(cmd)
        if run:
            execute(cmd,screen=verbose)


In [8]:
def parse_and_filter_clustering_table(INPATH,OUTPATH,sample_name,min_read_size,\
                                      min_subpools=3,relative_th_factor=0.05,\
                                      host_ref_file=False,host_id=0.95,\
                                      remove_files=False,threads=20,verbose=False,run=True):
    
    centroids_full_path = INPATH + sample_name + '_OTU.fna'
    table_full_path = INPATH + sample_name + '_OTU.txt'
    
    ensure_dir(OUTPATH)
    parsed_table_full_path = OUTPATH + sample_name + '_OTU.csv'


    ##########
    


    log('Parsing clustering information from %s...' % table_full_path )
    otu = pd.read_csv(table_full_path ,sep='\t',index_col=None,header=None)
    otu.columns  = ['type','cluster','length','ident','strand','','','align','q','h']

    log('Populating table features...')
    #Extract cluster sizes from type 'C' in otu table
    q = otu[otu['type'] == 'C']['q'].values
    clusterSize = otu[otu['type'] == 'C']['length'].values
    clusterSizeDict = dict(zip(q,clusterSize))

    #Only consider 'Hit' or 'Seed' rows 
    otu = otu[ (otu.type == 'H') | (otu.type == 'S') ]

    seeds = otu[otu.h == '*'].index
    otu.loc[seeds,'h'] = otu.loc[seeds,'q']

    log('Calculating cluster sizes...')
    otu['clusterSize'] = otu['h'].apply(lambda x: clusterSizeDict[x])

    otu.loc[:,'seed'] = otu['h'].apply(lambda x: x.split(';')[0])
    otu.loc[:,'seed'] = otu.loc[:,'seed']  + ';size=' +  otu.loc[:,'clusterSize'].astype(str) 

    #remove trailing ';' in 'size= ;'
    otu['q'] = otu.q.apply(lambda x: x[:-1] if x[-1] ==';' else x)
    otu['h'] = otu.h.apply(lambda x: x[:-1] if x[-1] ==';' else x)

    #add sequences from centroids file
    centroids_indexs = otu[otu['type'] == 'S'].index
    centroids_indexer = SeqIO.index(centroids_full_path, "fasta")
    otu.loc[centroids_indexs,'seq'] =  otu.loc[centroids_indexs,'seed'].apply(lambda x: str(centroids_indexer[x].seq))

    otu.loc[centroids_indexs,'domain'] = sample_name

    #otu['well'] = otu['q'].apply(lambda x: int(re.findall('\d{3}',x)[0]))
    otu['well'] = otu['q'].apply(lambda x: x.split('.')[0])
    


    #merge all cluster members present in a well 
    otu.loc[:,'readSize']= otu['q'].apply(lambda x: int(re.findall('size=(\d+)',x)[0]))
    frames = []
    for index,group in otu.groupby('seed'):
        d = group.groupby('well').first().reset_index()
#         d['readSize'] = group.groupby('well')['readSize'].agg('sum').values
#         frames.append(d)
#VL edit: only consider OTU if seed > 3 reads
        if d[d.type=='S'].readSize.values>=min_read_size:
            d['readSize'] = group.groupby('well')['readSize'].agg('sum').values
            frames.append(d)
    otu = pd.concat(frames).reset_index()
    cols_to_keep = [x for x in otu.columns if ('Unnamed') not in x]
    otu = otu[cols_to_keep]
    
    #Count number of wells apperances for each seed 
    clusterWellDict = otu.groupby('h').well.nunique().to_dict()
    otu['clusterWell'] = otu['h'].apply(lambda x: clusterWellDict[x])

    log('%s clusters found (%s OTUs)...' % (otu.h.nunique(),len(otu)))


    #min_read_size
    log('Dropping clusters with min_read_size < %s from table...' % (min_read_size,))
    drop_min_read = otu[otu['readSize'] < min_read_size].index
    otu.drop(drop_min_read,inplace=True)
    log('%s clusters found (%s OTUs)...' % (otu.h.nunique(),len(otu)))
    
    
    otu.to_csv(parsed_table_full_path)
    return otu

        

In [17]:
def calc_domain_occurances(filtered_clustering_table,MIN_PAIR_COUNT=3,verb=False): 
    Nwells = filtered_clustering_table['well'].nunique()

    if verb:
        print('Calculating pairs statistics...')
        print('%s subpools found...' % Nwells)

    #create a dic where key is well and values are all the seeds inside the well
    wells_dict = {} 
    for w,gr in filtered_clustering_table.groupby('well')['seed']:
        wells_dict[w] = gr.unique()
        
    clusterWellDict = filtered_clustering_table.groupby('seed').well.nunique().to_dict()

    #iterate over all wells and generate all pairwise combinations of seeds 
    #make a dic where key is the pair tuple the value is the number of co-occurance for this pair across plate 
    if verb:
        print('Counting pairs occurances... ')
    pairs_dict = defaultdict(list)

    if len(wells_dict.keys()) > Nwells: 
        print('Number of wells is larger than %s...' % Nwells)
        sys.exit()

    counter = 0
    for counter,well in enumerate(wells_dict.keys()):
        if ((counter % 96 == 0) & (verb) ):
            print('%s wells processed...' % counter)

        for pair in combinations(set(wells_dict[well]),2):
                pair = tuple(sorted(pair))
                pairs_dict[pair].append(well)

    if verb:
        print('Current pairs count %s' % len(pairs_dict))

    #Iterate over all pairs are remove those which appears together below / above cutoff counts
    #Removes most of pairs which appear together only once or twince in the entire plate 

    if verb:
        print('Removing pairs with MIN_PAIR_COUNT < %s' % MIN_PAIR_COUNT)
    
    l = len(pairs_dict)
    for i,k in enumerate(pairs_dict.keys()):
        if (len(pairs_dict[k]) < MIN_PAIR_COUNT) :
            del pairs_dict[k]
    if verb:
        print('%s pairs removed...' % (l-len(pairs_dict)))

    if verb:
        print('Current pairs count %s' % len(pairs_dict))
        print('Performing pair-wise Fisher test...')

    indexs = pairs_dict.keys()
    #Build df to hold all pairs and their binomial scores 
    df = pd.DataFrame(index=indexs,columns = ['V1','V2','Ov1','Ov2','P0','binom','count'])
    df['V1'] = [x[0] for x in pairs_dict.keys()]
    df['V2'] = [x[1] for x in pairs_dict.keys()]
    df['Ov1'] = df.V1.apply(lambda x: clusterWellDict[x])
    df['Ov2'] = df.V2.apply(lambda x: clusterWellDict[x])
    df['P0'] = [float(a*b)/(Nwells**2) for a,b in zip(df['Ov1'].values,df['Ov2'].values)] 
    df['wells'] = pairs_dict.values()

    df['count'] = df['wells'].apply(lambda x: len(x))

    df['a'] = df['count']
    df['b'] = df['Ov1'] - df['count']
    df['c'] = df['Ov2'] - df['count']
    df['d'] = Nwells- df['count']

    df['fisher'] = df.apply(lambda row: calc_fisher(row['a'],row['b'],row['c'],row['d']),axis=1).values
    df['pvalue'] = df.fisher.apply(lambda x: x[1] )
    df['odds'] = df.fisher.apply(lambda x: x[0] )
    df['P']  = df.pvalue.apply(lambda x: -np.log10(x))

    if verb:
        print('Fisher test done')
    
    return df



## 1) cluster amplicons into OTUs with usearch 

In [5]:
KSstack='/nasdata/Vincent/Strep/stacks/KS_reads_SORTED.fna'
ADstack='/nasdata/Vincent/Strep/stacks/AD_reads_SORTED.fna'
KSmegapool='/nasdata/Vincent/Strep/megapools/KS_reads_SORTED.fna'
ADmegapool='/nasdata/Vincent/Strep/megapools/AD_reads_SORTED.fna'
KSgenomes='/nasdata/Vincent/Strep/genomes/KS_genomes_reads_SORTED.fna'
ADgenomes='/nasdata/Vincent/Strep/genomes/AD_genomes_reads_SORTED.fna'

In [4]:
Dir='/nasdata/Vincent/Strep/All_amplicons/'

In [9]:
#KS: concatenate all source of reads, dereplicated and SORTED
remove_files=True
threads=50
run=True
verbose=False

OUTPATH=Dir
sample_name='3KS'
merged_output_file = OUTPATH +sample_name+'_SORTED.fna'
cmd = 'cat %s %s %s > %s' %(KSstack, KSmegapool, KSgenomes,merged_output_file)

execute(cmd,screen=verbose)
print 'done'

centroids_filename = OUTPATH + sample_name + '_OTU.fna'
table_filename = OUTPATH + sample_name + '_OTU.txt'
merged_sorted_output_file=OUTPATH +sample_name+'_SORTED.fna'

cluster_id=0.95

#cluster
input_file = merged_sorted_output_file
centroids_filename = OUTPATH + sample_name + '_OTU.fna'
table_filename = OUTPATH + sample_name + '_OTU.txt'

cmd = ('vsearch '
       '--threads %s ' 
       '--cluster_smallmem %s '
       '--id %s '
       '--iddef 1 '
       '--sizein '
       '--sizeout '
       '--centroids %s '
       '--uc %s'
       % (threads,input_file,cluster_id,centroids_filename,table_filename)
      )

if run:
    execute(cmd,screen=verbose)
    
print 'done'


done
done


In [35]:
#AD: concatenate all source of reads, dereplicated and SORTED

sample_name='3AD'
merged_output_file = OUTPATH +sample_name+'_SORTED.fna'
cmd = 'cat %s %s %s > %s' %(ADstack, ADmegapool, ADgenomes,merged_output_file)

execute(cmd,screen=verbose)
print 'done'

centroids_filename = OUTPATH + sample_name + '_OTU.fna'
table_filename = OUTPATH + sample_name + '_OTU.txt'
merged_sorted_output_file=OUTPATH +sample_name+'_SORTED.fna'

cluster_id=0.95

#cluster
input_file = merged_sorted_output_file
centroids_filename = OUTPATH + sample_name + '_OTU.fna'
table_filename = OUTPATH + sample_name + '_OTU.txt'

cmd = ('vsearch '
       '--threads %s ' 
       '--cluster_smallmem %s '
       '--id %s '
       '--iddef 1 '
       '--sizein '
       '--sizeout '
       '--centroids %s '
       '--uc %s'
       % (threads,input_file,cluster_id,centroids_filename,table_filename)
      )

if verbose:
    print('\n')
    print(cmd)

if run:
    execute(cmd,screen=verbose)
    
print 'done'

done


## 2) filter OTUs (cleanup)

In [19]:
sample_name='3KS'
INPUT_PATH = Dir
OUTPATH = Dir+ 'clustering/'
MIN_READ_SIZE=3

parse_and_filter_clustering_table(INPUT_PATH, OUTPATH, sample_name,threads=50,
                                                                      min_read_size=MIN_READ_SIZE,
                                                                      verbose=True)

sample_name='3AD'
INPUT_PATH = Dir
OUTPATH = Dir+ 'clustering/'
MIN_READ_SIZE=3

parse_and_filter_clustering_table(INPUT_PATH, OUTPATH, sample_name,threads=50,
                                                                      min_read_size=MIN_READ_SIZE,
                                                                      verbose=True)

In [5]:
OUTPATH = Dir+ 'clustering/'
AD_OTU=pd.read_csv(OUTPATH + '3AD_OTU.csv',index_col=0)
print len(AD_OTU)
KS_OTU=pd.read_csv(OUTPATH + '3KS_OTU.csv',index_col=0)
print len(KS_OTU)
AD_KS_merged_filtered_clustering_table=pd.concat([AD_OTU, KS_OTU], ignore_index=True)
AD_KS_merged_filtered_clustering_table['kind'] = AD_KS_merged_filtered_clustering_table.well.apply(lambda x: x.split('_')[0])
print len(AD_KS_merged_filtered_clustering_table)

63729
56489
120218


#### For each cluster, we remove sequences with low read counts (<5%) relative to the maximum read count of a sequence in the cluster, this is done independently for reads acquired in different MiSeq runs (well-pools / plate-pools / genomes) as the maximum read count observed for each cluster varies in each run. 

In [7]:
otu_wellOnly=AD_KS_merged_filtered_clustering_table[AD_KS_merged_filtered_clustering_table.kind=='well']
otu_megapoolOnly=AD_KS_merged_filtered_clustering_table[AD_KS_merged_filtered_clustering_table.kind=='Megapool']
otu_genomesOnly=AD_KS_merged_filtered_clustering_table[AD_KS_merged_filtered_clustering_table.kind=='StrepGenome']

In [8]:
print 'otu_wellOnly'
otu=otu_wellOnly

relative_th_factor=0.05
#relative_th_factor - size thresholding as function max size in biggest OTU
print('%s clusters found (%s OTUs)...' % (otu.h.nunique(),len(otu)))
print('Dropping reads with readSize < max_cluster_readsize*%s ...' % relative_th_factor)
groups = otu.sort_values('readSize').groupby('seed')
d = groups.apply(lambda g: g[ (g['readSize'] > (g['readSize'].max()* relative_th_factor)) | (g['type'] == 'S')])
otu = d.drop('seed',axis=1).reset_index().drop('level_1',axis=1)

otu_wellOnly=otu

print('%s clusters found (%s OTUs)...' % (otu_wellOnly.h.nunique(),len(otu)))

print ''
print 'otu_megapoolOnly'
otu=otu_megapoolOnly

relative_th_factor=0.05
#relative_th_factor - size thresholding as function max size in biggest OTU
print('%s clusters found (%s OTUs)...' % (otu.h.nunique(),len(otu)))
print('Dropping reads with readSize < max_cluster_readsize*%s ...' % relative_th_factor)
groups = otu.sort_values('readSize').groupby('seed')
d = groups.apply(lambda g: g[ (g['readSize'] > (g['readSize'].max()* relative_th_factor)) | (g['type'] == 'S')])
otu = d.drop('seed',axis=1).reset_index().drop('level_1',axis=1)

otu_megapoolOnly=otu

print('%s clusters found (%s OTUs)...' % (otu_megapoolOnly.h.nunique(),len(otu)))
print ''
print 'otu_genomesOnly'
otu=otu_genomesOnly

relative_th_factor=0.05
#relative_th_factor - size thresholding as function max size in biggest OTU
print('%s clusters found (%s OTUs)...' % (otu.h.nunique(),len(otu)))
print('Dropping reads with readSize < max_cluster_readsize*%s ...' % relative_th_factor)
groups = otu.sort_values('readSize').groupby('seed')
d = groups.apply(lambda g: g[ (g['readSize'] > (g['readSize'].max()* relative_th_factor)) | (g['type'] == 'S')])
otu = d.drop('seed',axis=1).reset_index().drop('level_1',axis=1)

otu_genomesOnly=otu


print('%s clusters found (%s OTUs)...' % (otu_genomesOnly.h.nunique(),len(otu)))

cleaned_merged_filtered_clustering_table=pd.concat([otu_wellOnly, otu_megapoolOnly,otu_genomesOnly], ignore_index=True, sort=False)

otu_wellOnly
9973 clusters found (82594 OTUs)...
Dropping reads with readSize < max_cluster_readsize*0.05 ...
9973 clusters found (47015 OTUs)...

otu_megapoolOnly
2981 clusters found (34035 OTUs)...
Dropping reads with readSize < max_cluster_readsize*0.05 ...
2981 clusters found (20590 OTUs)...

otu_genomesOnly
2102 clusters found (3589 OTUs)...
Dropping reads with readSize < max_cluster_readsize*0.05 ...
2102 clusters found (2668 OTUs)...


## 3) localize positions of OTUs in the library


In [13]:
## requires specific list of plates pooled in each stack 
stackList=[[52, 93, 57, 80, 105, 121, 66, 145, 23, 82, 38, 141, 146, 44, 24, 97, 99, 126, 150, 78, 56, 108, 18, 123, 149], [110, 130, 113, 71, 137, 39, 51, 28, 35, 89, 31, 84, 115, 107, 14, 6, 21, 69, 58, 64, 131, 125, 15, 27, 119], [85, 133, 1, 29, 40, 147, 3, 122, 49, 135, 128, 62, 143, 120, 88, 140, 32, 45, 26, 9, 87, 63, 75, 138, 134], [22, 142, 36, 4, 90, 43, 53, 129, 117, 30, 25, 13, 72, 83, 11, 2, 67, 42, 116, 74, 96, 76, 46, 41, 48], [92, 19, 8, 59, 112, 104, 33, 118, 100, 103, 70, 37, 47, 20, 34, 111, 54, 91, 86, 124, 10, 79, 81, 68, 61], [106, 73, 136, 77, 60, 101, 16, 95, 55, 5, 98, 114, 144, 65, 127, 109, 12, 132, 7, 139, 50, 17, 94, 148, 102]]
MPstackDict={}
for idx, stack in enumerate(stackList):
    for MP in stack:
        MPstackDict[MP]=idx+1  #query MP, get corresponding stack
        
wellstackDict={}
for i in range(1,2305):
    if i%384>0:
        wellstackDict[i]=i/384+1
    else:
        wellstackDict[i]=i/384
        

Cols=range(1,25)
Rows=[]
for letter in range(0,16):
    Rows.append(string.ascii_uppercase[letter])


def platePosition(wellNb):
    icol=(wellNb/16)+min(1,wellNb%16)
    irow=Rows[wellNb%16-1]
    return str(irow+str(icol))


OTUgaranteedLoc={}
OTU2choicesLoc={}

lost=[]
appeared=[]
unclearLoc=[]

for otu, df_group in cleaned_merged_filtered_clustering_table.groupby('seed'):
    flagMP=False
    flagwell=False
    flagGotcha=False
    seq=None
 
    
    #create a dataframe with 6 stacks as columns and 2 indexes: MP and wells
    OTUlocDF=pd.DataFrame(index=['MP','well'],columns=[1,2,3,4,5,6])
    OTUlocDF = OTUlocDF.astype(object)
    for row_index, row in df_group.iterrows():
        well=int(row.well.split('_')[1])

        if type(row.seq) is str:
            seq=row.seq
        if row.well.startswith('Megapool'):
            if well<151:
                flagMP=True
                try:
                    OTUlocDF.at['MP',MPstackDict[well]].append(well)
                except:
                    OTUlocDF.at['MP',MPstackDict[well]]=[well]
        elif row.well.startswith('well'):
            flagwell=True
            try:
                OTUlocDF.at['well',wellstackDict[well]].append(platePosition(well-384*(wellstackDict[well]-1)))
            except:
                OTUlocDF.at['well',wellstackDict[well]]=[platePosition(well-384*(wellstackDict[well]-1))]
   # print OTUlocDF

    #find stacks with 1 MP and 1 well:
    for i in range(1,7):
        try:
            if len(OTUlocDF.loc['MP',i])==1:
                if len(OTUlocDF.loc['well',i])==1:
                    flagGotcha=True
                    location=str(OTUlocDF.loc['MP',i][0])+'_'+str(OTUlocDF.loc['well',i][0])
                    if otu in OTUgaranteedLoc:
                        OTUgaranteedLoc[otu].append(location)
                    else:
                        OTUgaranteedLoc[otu]=[location]
                        OTU2choicesLoc.pop(otu, None)
            elif len(OTUlocDF.loc['MP',i])*len(OTUlocDF.loc['well',i])<=4:
                flagGotcha=True
                possibleLocs=list(itertools.product(OTUlocDF.loc['MP',i], OTUlocDF.loc['well',i]))
                possibleLocs=[str(x[0])+'_'+str(x[1]) for x in possibleLocs]
                if otu in OTU2choicesLoc:
                    OTU2choicesLoc[otu].append(possibleLocs)
                elif otu not in OTUgaranteedLoc:
                    OTU2choicesLoc[otu]=[possibleLocs]
        except:
            continue
            
    if flagMP and not flagwell:
        lost.append(otu)
        
    elif flagwell and not flagMP:
        appeared.append(otu)

    elif flagwell and flagMP and not flagGotcha:
        unclearLoc.append(otu)

print 'lost '+str(len(lost))
print 'appeared '+str(len(appeared))
print 'unclear '+str(len(unclearLoc))
print 'OTUgaranteedLoc '+str(len(OTUgaranteedLoc))
print 'OTU2choicesLoc '+str(len(OTU2choicesLoc))

locsDict={}
for i in OTU2choicesLoc:
    locsDict[i]=OTU2choicesLoc[i]
for i in OTUgaranteedLoc:
    locsDict[i]=OTUgaranteedLoc[i]

In [10]:
seedDF=cleaned_merged_filtered_clustering_table[cleaned_merged_filtered_clustering_table.type=='S']

#create a dic where key is well and values are all the seeds inside the well
wells_dict = {} 
for w,gr in cleaned_merged_filtered_clustering_table.groupby('seed')['well']:
    wells_dict[w] = gr.unique()

seedDF['found']= seedDF.seed.apply(lambda x: wells_dict[x])

def uniqueGenome(found):
    G=[]
    for i in found:
        if i.startswith('StrepGenome'):
            G.append(i)
    if len(set(G))==1:
        return G[0]
    elif len(set(G))==0:
        return 'unknown'
    else:
        return 'multiple'
    

def genomes(found):
    G=[]
    for i in found:
        if i.startswith('StrepGenome'):
            G.append(i)
    return len(set(G))

def wells(found):
    W=[]
    for i in found:
        if i.startswith('well'):
            W.append(i)
    return len(set(W))

def megapools(found):
    M=[]
    for i in found:
        if i.startswith('Megapool'):
            M.append(i)
    return len(set(M))

def megapoolsList(found):
    M=[]
    for i in found:
        if i.startswith('Megapool'):
            M.append(i.split('_')[1])
    return str(M)

def location(seed):
    if seed in locsDict:
        return str(locsDict[seed])
    elif seed in unclearLoc:
        return 'unclear'
    elif seed in appeared:
        return 'appeared'
    else:
        return 'lost'

seedDF['uniqueGenome']= seedDF.found.apply(lambda x: uniqueGenome(x))
seedDF['genomes']= seedDF.found.apply(lambda x: genomes(x))
seedDF['wells']= seedDF.found.apply(lambda x: wells(x))
seedDF['megapools']= seedDF.found.apply(lambda x: megapools(x))
seedDF['megapoolsList']= seedDF.found.apply(lambda x: megapoolsList(x))
seedDF['location']= seedDF.seed.apply(lambda x: location(x))
seedDF['network']='none'
seedDF['network_tag']='none'

## 4) evaluate co-occurrences and connect domains into networks 

### a. evaluate co-occurrences between biosynthetic domains in each category of pools

In [21]:
#1) co-occurrences in well-pools:
domain_occurances_table = calc_domain_occurances(otu_wellOnly ,MIN_PAIR_COUNT=1, verb=True)

otu_wellOnly.set_index('seed', inplace=True)
seedDF.set_index('seed', inplace=True)
seedDF['seed']=seedDF.index
otu_wellOnly['uniqueGenome']=seedDF['uniqueGenome']
otu_wellOnly['location']=seedDF['location']
otu_wellOnly['megapoolsList']=seedDF['megapoolsList']
otu_wellOnly['seed']=otu_wellOnlyTest.index
otu_wellOnly['seq']=seedDF['seq']

#2) co-occurrences in genomes:
domain_occurances_tableGenome= calc_domain_occurances(otu_genomesOnly ,MIN_PAIR_COUNT=1, verb=True)

#3) co-occurrences in plate-pools:
domain_occurances_tableMP = calc_domain_occurances(otu_megapoolOnly ,MIN_PAIR_COUNT=1, verb=True)

#4) combine all 3 co-occurrences:
domain_occurances_table_combined=domain_occurances_table.copy()
domain_occurances_table_combined['combined_locations']=domain_occurances_table['wells'].copy()
domain_occurances_table_combined['pvalueMP']=1
domain_occurances_table_combined['pvalueGenomes']=1

idx = domain_occurances_table.index.intersection(domain_occurances_tableMP.index)
domain_occurances_table_combined.loc[idx,'pvalueMP']=domain_occurances_tableMP.loc[idx,'pvalue']
domain_occurances_table_combined.loc[idx,'combined_locations']=domain_occurances_table_combined.loc[idx,'combined_locations']+domain_occurances_tableMP.loc[idx,'wells']
idx = domain_occurances_table.index.intersection(domain_occurances_tableGenome.index)
domain_occurances_table_combined.loc[idx,'pvalueGenomes']=domain_occurances_tableGenome.loc[idx,'pvalue']
domain_occurances_table_combined['combined_pvalue']=domain_occurances_table_combined['pvalue']*domain_occurances_table_combined['pvalueMP']*domain_occurances_table_combined['pvalueGenomes']

def multireps(wells):
    score=0
    plateLocs=[]   
    for w in wells:   
        if w.startswith('well'):
            plateLocs.append(int(w.split('_')[1]))
        elif w.startswith('Strep'):
            score+=1
        elif w.startswith('Mega'):
            score+=1
    if max(plateLocs)-min(plateLocs)>384:
        score+=1
    return score

domain_occurances_table_combined['multireplicate'] = domain_occurances_table_combined.apply(lambda x: multireps(x['combined_locations']),axis=1).values
domain_occurances_table_final = domain_occurances_table_combined[(domain_occurances_table_combined['multireplicate']>=1)]

Calculating pairs statistics...
97 subpools found...
Counting pairs occurances... 
0 wells processed...
96 wells processed...
Current pairs count 53360
Removing pairs with MIN_PAIR_COUNT < 1
0 pairs removed...
Current pairs count 53360
Performing pair-wise Fisher test...
Fisher test done


### b. apply threshold on pvalue and generate graph

In [29]:
alpha = 10**-5
beta=30

method = 'pvalue'
networkFile = Dir+ 'Combined_Multigenome_networks_' + str(alpha) + '_' + str(beta)+ '_'  + str(method) +'3.graphml'
filtered_clustering_table=otu_wellOnly.copy()
G=nx.Graph()
pairs_occurances=domain_occurances_table_final[domain_occurances_table_final['odds'] > beta]
pairs_occurances['pair'] = zip(pairs_occurances.V1.values,pairs_occurances.V2.values)
if method == 'pvalue':
    try:
        edges = pairs_occurances[pairs_occurances['combined_pvalue'] < alpha]['pair'].apply(lambda x: ast.literal_eval(x)).values
    except:
        edges = pairs_occurances[pairs_occurances['combined_pvalue'] < alpha]['pair'].values
    finally:
        risks = pairs_occurances[pairs_occurances['combined_pvalue'] < alpha]['combined_pvalue'].apply(lambda x: -np.log10(x)).astype(str).values
        weightedEdges = [ e + (b,) for e,b  in zip(edges,risks)]
else:
    (reject, pvals_correct,a,b) = multipletests(pairs_occurances.pvalue.values,alpha,method)
    pairs_occurances['pvalue_correct'] = pvals_correct
    pairs_occurances['reject'] = reject
    try:
        edges = pairs_occurances[pairs_occurances['reject']]['pair'].apply(lambda x: ast.literal_eval(x)).values
    except:
        edges = pairs_occurances[pairs_occurances['reject']]['pair'].values
    finally:
        risks = pairs_occurances[pairs_occurances['reject']]['pvalue_correct'].apply(lambda x: -np.log10(x)).astype(str).values
        weightedEdges = [ e + (b,) for e,b  in zip(edges,risks)]

G.add_weighted_edges_from(weightedEdges)

log("Constructing network --> %s %s" % (method,alpha))
log("%s nodes and %s edges found..." % (len(G.nodes()),len(G.edges)))


wellsReadsDict = filtered_clustering_table.astype(str).groupby('seed')['well'].apply( lambda x: set(x.tolist())).to_dict()
s = pd.Series(wellsReadsDict)
attr_dict = s.apply(lambda x: '_'.join(sorted(list(x)))).to_dict()
nx.set_node_attributes(G, name='well', values=attr_dict)

#extract attributes from filtered_clustering_table to graph
for attr in ['seq','clusterSize','domain','uniqueGenome','location','megapoolsList']:
    attr_dict = dict(zip(filtered_clustering_table.loc[:,'seed'],filtered_clustering_table.loc[:,attr]))
    nx.set_node_attributes(G, name=attr, values=attr_dict)

print str(nx.number_of_nodes(G))+" nodes"
print str(nx.number_of_edges(G))+" edges"
print str(nx.number_connected_components(G))+" networks"

/home/vlibis/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Constructing network --> pvalue 1e-05
2805 nodes and 17864 edges found...


/home/vlibis/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:40: FutureWarning: 'seed' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version


2805 nodes
17864 edges
291 networks


### c. cleanup graph

In [170]:
G_clean=G.copy()
## clean nodes found in unreasonably high number of wells 
removeList=[]
for nd in G_clean.nodes(): 
    nbWells=len(G_clean.node[nd]['well'].split('well_'))
    if nbWells>100:
        removeList.append(nd)
G_clean.remove_nodes_from(removeList)  

def edgeFragility(Subgraph,Edge): #check if two components are formed when removing an edge   
    testGraph=Subgraph.copy()
    testGraph.remove_edge(Edge[0],Edge[1])
    if nx.number_connected_components(testGraph)>1:
        #check that components are made of several nodes
        #removing 1 edge can at most create 2 fragments
        for fragment in nx.connected_component_subgraphs(testGraph, copy=False): 
            if fragment.number_of_nodes()<3:
                return False          
        return True
    else:
        return False

def nodeFragility(Subgraph,Node): #check if two components are formed when removing a node
    testGraph=Subgraph.copy()
    testGraph.remove_node(Node)
    if nx.number_connected_components(testGraph)==1:
        return False 
    else:
        #check that at least 2 components are made of several nodes
        #removing 1 node can create more than 2 fragments
        largeChuncks=0
        for fragment in nx.connected_component_subgraphs(testGraph, copy=False): 
            if fragment.number_of_nodes()>2:
                largeChuncks+=1
        if largeChuncks>1:          
            return True
        return False


    
##Edges
removing=True  
while removing:
    removing=False 
    for subgraph in nx.connected_component_subgraphs(G_clean, copy=False):   
        if subgraph.number_of_nodes()>10:
            centrality=nx.edge_betweenness_centrality(subgraph, k=None, normalized=True, weight=None, seed=None)
            #remove most central edges if they connect two distinct communities
            for k,v in centrality.items():
                if v>0.05:
                    if edgeFragility(subgraph,k):
                        print 'removing '+str(k)
                        print ''
                        G_clean.remove_edge(k[0],k[1])
                        removing=True  

##Nodes


removing=True  
while removing:
    removing=False 
    removeList=[]
    for subgraph in nx.connected_component_subgraphs(G_clean, copy=False): 
        if subgraph.number_of_nodes()>10:
            node_centrality=nx.betweenness_centrality(subgraph, k=None, normalized=True, weight=None, endpoints=False, seed=None)
            #nodes sorted by decreasing centrality, check if removing one node breaks the subgraph
            sorted_nodes = sorted(node_centrality.items(), key=operator.itemgetter(1),reverse=True)            
            for k in sorted_nodes:
                if nodeFragility(subgraph,k[0]):
                    print 'removing node '+str(k[0])
                    print ''
                    removeList.append(k[0])
                    removing=True
    G_clean.remove_nodes_from(removeList)

print str(nx.number_of_nodes(G_clean))+" nodes"
print str(nx.number_of_edges(G_clean))+" edges"
print str(nx.number_connected_components(G_clean))+" networks"

removing ('well_0188.M03834:162:000000000-CYRCK:1:1101:7331:14218;size=64', 'well_1156.M03834:162:000000000-CYRCK:1:1101:14140:2719;size=22789')

removing ('well_0188.M03834:162:000000000-CYRCK:1:1101:7331:14218;size=64', 'well_0188.M03834:162:000000000-CYRCK:1:1101:21651:3291;size=9705')

removing ('well_0188.M03834:162:000000000-CYRCK:1:1101:7331:14218;size=64', 'well_0188.M03834:162:000000000-CYRCK:1:1101:8205:2362;size=11033')

removing ('well_0188.M03834:162:000000000-CYRCK:1:1101:7331:14218;size=64', 'well_0618.M03834:162:000000000-CYRCK:1:1101:5561:9353;size=1243')

removing ('well_0397.M03834:162:000000000-CYRCK:1:1103:4966:20365;size=205', 'well_0397.M03834:162:000000000-CYRCK:1:1115:12123:10706;size=36')

removing ('well_0296.M03834:162:000000000-CYRCK:1:1101:7351:4761;size=8744', 'well_1484.M03834:162:000000000-CYRCK:1:1101:15472:1958;size=19006')

removing ('well_1898.M03834:162:000000000-CYRCK:1:1101:18452:3823;size=3105', 'well_1484.M03834:162:000000000-CYRCK:1:1101:15472

### d. give numbers to each nodes and networks and create a fasta file for each network (all translation frames)

In [54]:
nx.set_node_attributes(G_clean,'0','subgraphTagNumber')
dict_subgraphTagNumber={}
lenNetworksDict={}
netNumber=0
for subgraph in nx.connected_component_subgraphs(G_clean, copy=False):     
    netNumber+=1
    netName=str(netNumber)  
    Dir='/nasdata/Vincent/Strep/All_amplicons/'
    subgraphFasta= Dir+'networkFasta_combined/'+netName+'_domains.faa'
    tagNumber=0
    with open(subgraphFasta,'wb') as outfile:
        for n in subgraph.nodes(): 
            size=nx.number_of_nodes(subgraph)
            tagNumber+=1
            tagName='>'+netName+"_size"+str(size)+'_#'+str(tagNumber)
            seedDF.loc[n,'network']=netName
            seedDF.loc[n,'network_tag']=tagName

            frames=['a','b','c']
            for idx,f in enumerate(frames):
                
                coding_dna=G_clean.node[n]['seq']
                coding_dna=coding_dna[idx:]
                coding_dna=Seq(coding_dna, generic_dna)
                proteinSeq=coding_dna.translate(to_stop=True)
                if len(proteinSeq) > 60:
                    spamwriter = csv.writer(outfile,delimiter=',')
                    spamwriter.writerow([tagName+f])
                    spamwriter.writerow([proteinSeq])

            #update graphml file with domain names and numbers
            dict_subgraphTagNumber[n]=tagName
            lenNetworksDict[netName]=tagNumber

nx.set_node_attributes(G_clean, name='subgraphTagNumber',values=dict_subgraphTagNumber)

#concatenate all fasta files into .fna
os.chdir(Dir+'networkFasta_combined/')
subprocess.call("cat *.faa > multigenomeNets.fna", shell=True)


nx.write_graphml(G_clean, networkFile)

print('Done!')

Done!


## 5) Annotate nodes with pblast (identity to known BGCs)

### a. Extract protein sequences from reference BGCs 

In [ ]:
mibigFolder='/nasdata/Vincent/Strep/mibig_gbk_2.0/'

def extractProteins(genbankFile):
    with open(genbankFile, "rU") as input_handle:
        with open(genbankFile.replace('.gbk','.faa') , "w") as output_handle:
            CDScounter=0
            sequences = SeqIO.parse(input_handle, "genbank") 
            for record in sequences:
                record.id=record.description.split(' ')[0]+"_"+record.id
                record.description=record.id
                for seq_feature in record.features :
                    if seq_feature.type=="CDS" :
                        if len(seq_feature.qualifiers['translation'][0])>180:
                            CDScounter+=1
                            output_handle.write(">%s_%s\n%s\n" % (
                                   record.id,
                                   str(CDScounter),
                                   seq_feature.qualifiers['translation'][0]))
                            
if not os.path.exists(mibigFolder+'MIBiG.fnaa'):               
    for filename in os.listdir(mibigFolder):
        if filename.endswith(".gbk"):
            if not os.path.exists(mibigFolder+filename.replace('.gbk','.faa')):
                extractProteins(mibigFolder+filename)
    
    os.chdir(mibigFolder)
    subprocess.call("cat *.faa > MIBiG.fnaa", shell=True)
 

### b. blastp all translated domains

In [ ]:
distancedir='/nasdata/Vincent/Strep/Distance/'

def checkBlast(distancedir,RESULT_FILE,QUERY_FILE,DB_INFILE,DB_OUTFILE):
    if not os.path.exists(DB_OUTFILE+'.pin'):
        subprocess.call(["makeblastdb -in "+DB_INFILE+" -dbtype prot -out "+DB_OUTFILE], shell= True)    
    if os.path.exists(distancedir+'blastWorks/'+RESULT_FILE):
        print "already blasted "+RESULT_FILE[:-4]
    else:
        print '-Blasting '+RESULT_FILE[:-4]+'...'
        subprocess.call(["blastp -query "+QUERY_FILE+" -db "+DB_OUTFILE+" -outfmt '10 qseqid sseqid evalue bitscore length pident sstart send' -out blastWorks/"+RESULT_FILE+" -max_target_seqs 2000000 -evalue 1e-20 -qcov_hsp_perc 80 -num_threads 56"], shell=True)

os.chdir(distancedir)

QUERY_FILE='/nasdata/Vincent/Strep/All_amplicons/networkFasta_combined/multigenomeNets.fna' 

#blast soil against MIBiG    
DB_OUTFILE=distancedir+'blastWorks/blastDBs/MIBiG_DB'
DB_INFILE=mibigFolder+'MIBiG.fnaa'
RESULT_FILE='multigenomes-VS-MIBiGDB.csv'
checkBlast(distancedir,RESULT_FILE,QUERY_FILE,DB_INFILE,DB_OUTFILE)

print ''
print 'done'

### c. read blast results and annotate graph file

In [141]:
blastFile='/nasdata/Vincent/Strep/Distance/blastWorks/multigenomes-VS-MIBiGDB.csv'
blast = pd.read_csv(blastFile,index_col=None,header=None)

blast.columns = ['qseqid', 'sseqid', 'evalue', 'bitscore', 'length', 'pident', 'sstart', 'send']
blast.sort_values(by='bitscore',ascending=False, inplace=True)  #sort by best Bitscore
blast.reset_index(drop=True, inplace=True)   
print str(blastFile.split('/')[-1])+': '+str(len(blast))+" blast hits"

seedDF.loc[:,'AA_id']=74
seedDF.loc[:,'top_MIBIG_protein']='None'

seedDF.set_index('network_tag', inplace=True)

print len(blast.groupby('qseqid'))
n=0
for otu,group in blast.groupby('qseqid'):
    bestHit=('','',0)
    n+=1
    if n%100==0:
        print n
    for row_index, row in group.iterrows():  
        if bestHit[2]==0:
            bestHit=(row.sseqid,row.sstart,row.pident)
    if bestHit[2]>74:  
        seedDF.loc[str(">"+otu[:-1]),'AA_id']=bestHit[2]
        seedDF.loc[str(">"+otu[:-1]),'top_MIBIG_protein']=bestHit[0]+'_'+str(bestHit[1])

seedDF['network_tag']=seedDF.index
seedDF.set_index('seed', inplace=True)
seedDF['seed']=seedDF.index

#extract attributes from filtered_clustering_table to graph
for attr in ['AA_id','top_MIBIG_protein']:
    attr_dict = dict(zip(seedDF.loc[:,'seed'],seedDF.loc[:,attr]))
    nx.set_node_attributes(G_clean, name=attr, values=attr_dict)
networkFileOut=Dir+"Combined_Multigenome_networks_annotated_recoveredOCTOBER.graphml"
nx.write_graphml(G_clean, networkFileOut)

multigenomes-VS-MIBiGDB.csv: 1535115 blast hits
1970
100
200
300
400
500
600
700
800
900
1100
1200
1300
1400
1500
1600
1700
1800
1900


/home/vlibis/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/vlibis/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
